In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
import joblib

# 1. Load the trained model and scaler
model_path = r"D:\FCDS\semster 8\IOT 2\Projects\project 2\tinyml-weather-classification\models\tensorflow\weather_model.h5"
scaler_path = r"D:\FCDS\semster 8\IOT 2\Projects\project 2\tinyml-weather-classification\models\tensorflow\scaler.pkl"
data_path = r"D:\FCDS\semster 8\IOT 2\Projects\project 2\tinyml-weather-classification\data\labeled\weather_data.csv"

# Load model and scaler
model = load_model(model_path)
scaler = joblib.load(scaler_path)

# Load and preprocess the training data
df = pd.read_csv(data_path)
X = df[['temperature', 'humidity']].values
X_train = scaler.transform(X)  # Apply the same scaling used during training

# Extract scaler parameters for ESP32 code
print("Scaler mean:", scaler.mean_)
print("Scaler scale (1/std):", scaler.scale_)
print("\nNote: Update these values in the ESP32 code")

# 2. Convert to TensorFlow Lite (float32)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the float32 model
tflite_float_path = r"D:\FCDS\semster 8\IOT 2\Projects\project 2\tinyml-weather-classification\models\tflite\weather_classifier.tflite"
with open(tflite_float_path, 'wb') as f:
    f.write(tflite_model)
print(f"Float32 TFLite model saved to {tflite_float_path}")

# 3. Create a quantized model (int8 quantization)
def representative_data_gen():
    # Use scaled training data as representative dataset
    for i in range(min(100, len(X_train))):  # Use first 100 samples or less if dataset is smaller
        yield [X_train[i:i+1].astype(np.float32)]

converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

quantized_tflite_model = converter.convert()

# Save the quantized model
quantized_path = r"D:\FCDS\semster 8\IOT 2\Projects\project 2\tinyml-weather-classification\models\tflite\weather_classifier_quantized.tflite"
with open(quantized_path, 'wb') as f:
    f.write(quantized_tflite_model)
print(f"Quantized TFLite model saved to {quantized_path}")

# Compare model sizes
import os
float_size = os.path.getsize(tflite_float_path)
quant_size = os.path.getsize(quantized_path)

print(f"\nModel size comparison:")
print(f"Float32 model: {float_size} bytes")
print(f"Quantized model: {quant_size} bytes")
print(f"Reduction: {(1 - quant_size/float_size)*100:.2f}%")

Scaler mean: [9.70352419 0.72762099]
Scaler scale (1/std): [11.22387586  0.21171362]

Note: Update these values in the ESP32 code
INFO:tensorflow:Assets written to: C:\Users\mazen\AppData\Local\Temp\tmpybxqza83\assets


INFO:tensorflow:Assets written to: C:\Users\mazen\AppData\Local\Temp\tmpybxqza83\assets


Float32 TFLite model saved to D:\FCDS\semster 8\IOT 2\Projects\project 2\tinyml-weather-classification\models\tflite\weather_classifier.tflite
INFO:tensorflow:Assets written to: C:\Users\mazen\AppData\Local\Temp\tmprlpi9w10\assets


INFO:tensorflow:Assets written to: C:\Users\mazen\AppData\Local\Temp\tmprlpi9w10\assets


Quantized TFLite model saved to D:\FCDS\semster 8\IOT 2\Projects\project 2\tinyml-weather-classification\models\tflite\weather_classifier_quantized.tflite

Model size comparison:
Float32 model: 2352 bytes
Quantized model: 2584 bytes
Reduction: -9.86%


C:\Users\mazen\AppData\Roaming\Python\Python39\site-packages\tensorflow\lite\python\convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


In [3]:
# 4. Convert TFLite model to C array
def convert_to_c_array(tflite_path, output_path, array_name='model'):
    with open(tflite_path, 'rb') as f:
        model_content = f.read()
    
    hex_array = ', '.join([f'0x{byte:02x}' for byte in model_content])
    c_array = f"const unsigned char {array_name}[] = {{\n  {hex_array}\n}};\n"
    c_array += f"const unsigned int {array_name}_len = {len(model_content)};"
    
    with open(output_path, 'w') as f:
        f.write(c_array)
    print(f"C array saved to {output_path}")

# Convert both models using your exact paths
convert_to_c_array(
    r"D:\FCDS\semster 8\IOT 2\Projects\project 2\tinyml-weather-classification\models\tflite\weather_classifier.tflite",
    r"D:\FCDS\semster 8\IOT 2\Projects\project 2\tinyml-weather-classification\firmware\src\weather_model_float.h",
    'model_float'
)

convert_to_c_array(
    r"D:\FCDS\semster 8\IOT 2\Projects\project 2\tinyml-weather-classification\models\tflite\weather_classifier_quantized.tflite",
    r"D:\FCDS\semster 8\IOT 2\Projects\project 2\tinyml-weather-classification\firmware\src\weather_model_quant.h",
    'model_quant'
)

print("\nC array conversion complete. Models ready for ESP32 deployment.")

C array saved to D:\FCDS\semster 8\IOT 2\Projects\project 2\tinyml-weather-classification\firmware\src\weather_model_float.h
C array saved to D:\FCDS\semster 8\IOT 2\Projects\project 2\tinyml-weather-classification\firmware\src\weather_model_quant.h

C array conversion complete. Models ready for ESP32 deployment.
